In [31]:
#Importing packages and checking GPU
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchsummary import summary 

if torch.cuda.is_available():
	print("PyTorch is using the GPU")
	GPUCount = torch.cuda.device_count()
	print(f"Found {GPUCount} GPUs")

	for i in range(GPUCount):
		print(f"GPU {i} found: {torch.cuda.get_device_name(i)}")

	device = torch.device("cuda:0")
else:
	print("PyTorch is using the CPU")
	device = torch.device("cpu")

print(f"Selected Device: {device}")

PyTorch is using the GPU
Found 1 GPUs
GPU 0 found: NVIDIA GeForce RTX 5070 Laptop GPU
Selected Device: cuda:0


In [32]:
#Load BuiltIn dataset
Transform = transforms.Compose(
	[transforms.ToTensor(),
  transforms.Normalize((0.5,),(0.5,))])
Transform = transforms.Compose(
	[transforms.ToTensor(),
  transforms.Normalize((0.5,),(0.5,))])

TrainSet = torchvision.datasets.EMNIST(root='./data',train=True,download=True,transform=Transform,split="letters",target_transform=lambda y:y-1)
TrainLoader = DataLoader(TrainSet, batch_size=128, shuffle=True)

TestSet = torchvision.datasets.EMNIST(root='./data',train=False,download=True,transform=Transform,split="letters",target_transform=lambda y:y-1)
TestLoader = DataLoader(TestSet, batch_size=128, shuffle=False)

print(f"Training Set size: {len(TrainSet)} | Test Set size: {len(TestSet)}")

Training Set size: 124800 | Test Set size: 20800


In [33]:
#Creating Model

#Define architecture
class EMNISTModel(nn.Module):
	def __init__(self):
		super(EMNISTModel,self).__init__()

		#Layer1
		self.conv1 = nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,padding=1)#Size does not change
		self.relu1 = nn.ReLU()
		self.pool1 = nn.MaxPool2d(kernel_size=2)#Size halves into 14x14

		#Layer2
		self.conv2 = nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,padding=1)
		self.relu2 = nn.ReLU()
		self.pool2 = nn.MaxPool2d(kernel_size=2)#Size halves into 7x7

		#Layer3
		self.conv3 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding=1)

		#Layer4
		self.conv4 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1)

		#Layer5
		self.flatten = nn.Flatten()

		#Layer6
		self.fc1 = nn.Linear(in_features=64*7*7,out_features=64)
		self.relu3 = nn.ReLU()

		#DropoutLayer
		self.dropout = nn.Dropout(p=0.2)

		#Layer7
		self.fc2 = nn.Linear(in_features=64,out_features=26)

	def forward(self,x):

		#Pass through Layer1
		x = self.pool1(self.relu1(self.conv1(x)))

		#Pass through Layer2
		x = self.pool2(self.relu2(self.conv2(x)))

		#Pass through Layer3
		x = self.conv3(x)

		#Pass through Layer4
		x = self.conv4(x)

		#Pass through Layer5
		x = self.flatten(x)

		#Pass through Layer6
		x = self.relu3(self.fc1(x))

		#Pass through DropoutLayer
		x = self.dropout(x)

		#Pass through Layer7
		x = self.fc2(x)

		#Return Prediction
		return x

#Create and print summary	
print("Creating EMNIST Model...")
model = EMNISTModel().to(device)
print("Model Created")

print("Model Summary: ")
summary(model,input_size=(1,28,28))

#Compile model
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Creating EMNIST Model...
Model Created
Model Summary: 
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
              ReLU-2           [-1, 32, 28, 28]               0
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4           [-1, 32, 14, 14]           9,248
              ReLU-5           [-1, 32, 14, 14]               0
         MaxPool2d-6             [-1, 32, 7, 7]               0
            Conv2d-7             [-1, 64, 7, 7]          18,496
            Conv2d-8             [-1, 64, 7, 7]          36,928
           Flatten-9                 [-1, 3136]               0
           Linear-10                   [-1, 64]         200,768
             ReLU-11                   [-1, 64]               0
          Dropout-12                   [-1, 64]               0
           Linear-13                   [-1, 26] 

In [34]:
#Training and saving the model

print("Training Model...")
epochs = 100
patience = 5
counter = 0
MaxAccuracy = float('-inf')

for epoch in range(epochs):
	#Train Loop

	#Set model to training mode
	model.train()
	TrainLoss = 0.0

	for i,data in enumerate(TrainLoader,0):
		inputs,labels = data[0].to(device),data[1].to(device)

		#Zero the parameter gradients
		optimizer.zero_grad()

		#Forward pass
		outputs = model(inputs)

		#Calculate loss
		loss = loss_function(outputs,labels)

		#Backward pass
		loss.backward()

		#Update weights
		optimizer.step()

		#Update loss
		TrainLoss += loss.item()
	
	#Validation Loop
	model.eval()
	ValidationLoss = 0.0
	correct = 0
	total = 0

	with torch.no_grad():
		for data in TestLoader:
			images,labels = data[0].to(device),data[1].to(device)
			outputs = model(images)
			loss = loss_function(outputs,labels)
			ValidationLoss += loss.item()

			_,predicted = torch.max(outputs.data,1)
			total += labels.size(0)
			correct += (predicted == labels).sum().item()

		acc = 100*correct/total
		print(f"Epoch: {epoch+1}/{epochs} | "
			f"Training Loss: {TrainLoss/len(TrainLoader):.3f} | "
			f"Validation Loss: {ValidationLoss / len(TestLoader):.3f} | "
			f"Accuracy: {acc:.3f}%")
		
		if acc > MaxAccuracy:
			MaxAccuracy = acc
			counter = 0
			torch.save(model.state_dict(),"EMNISTModel.pth")
			print(f"Accuracy has improved at epoch {epoch+1}. Model Saved!")
		else:
			counter += 1
			print(f"Accuracy has not improved. Patience:{counter}/{patience}")
		
		if counter >= patience:
			print("Early stopping triggered!")
			break
		
print("Training Complete")


Training Model...
Epoch: 1/100 | Training Loss: 0.608 | Validation Loss: 0.284 | Accuracy: 90.029%
Accuracy has improved at epoch 1. Model Saved!
Epoch: 2/100 | Training Loss: 0.333 | Validation Loss: 0.240 | Accuracy: 92.298%
Accuracy has improved at epoch 2. Model Saved!
Epoch: 3/100 | Training Loss: 0.291 | Validation Loss: 0.239 | Accuracy: 92.106%
Accuracy has not improved. Patience:1/5
Epoch: 4/100 | Training Loss: 0.266 | Validation Loss: 0.217 | Accuracy: 93.149%
Accuracy has improved at epoch 4. Model Saved!
Epoch: 5/100 | Training Loss: 0.249 | Validation Loss: 0.214 | Accuracy: 92.995%
Accuracy has not improved. Patience:1/5
Epoch: 6/100 | Training Loss: 0.239 | Validation Loss: 0.215 | Accuracy: 93.130%
Accuracy has not improved. Patience:2/5
Epoch: 7/100 | Training Loss: 0.230 | Validation Loss: 0.217 | Accuracy: 93.014%
Accuracy has not improved. Patience:3/5
Epoch: 8/100 | Training Loss: 0.219 | Validation Loss: 0.203 | Accuracy: 93.495%
Accuracy has improved at epoch 8.

In [35]:
#Testing Model

print("Testing model with inbuilt dataset...")

model.load_state_dict(torch.load("EMNISTModel.pth"))

TestLoss = 0.0
correct = 0
total = 0

with torch.no_grad():
	for images, labels in TestLoader:

		#Move data to device
		images,labels = images.to(device),labels.to(device)

		#Forward pass
		outputs = model(images)

		#Calculate loss
		loss = loss_function(outputs,labels)
		TestLoss += loss.item()

		#Get predicted class
		_,predicted = torch.max(outputs.data,1)

		#Update total and correct counts
		total += labels.size(0)
		correct += (predicted == labels).sum().item()
	
#Test Results

FinalLoss = TestLoss/len(TestLoader)
FinalAcc = 100*correct/total

print(f"Test Accuracy: {FinalAcc:.3f}%")
print(f"Test Loss: {FinalLoss:.3f}")

Testing model with inbuilt dataset...
Test Accuracy: 93.760%
Test Loss: 0.200
